In [1]:
from imprint.nb_util import setup_nb
setup_nb()

[worker_id=None] 2023-03-10 08:59:44,897 - imprint - DEBUG 
Enabling 64-bit floats in JAX.


In [2]:
import os
import coiled
import imprint 
import confirm
from distributed.diagnostics.plugin import UploadDirectory

coiled.create_software_environment(name="confirm-coiled", conda="env-coiled.yml")
cluster = coiled.Cluster(
    name="confirm-coiled",
    software="confirm-coiled",
    n_workers=1,
    worker_vm_types=["g4dn.xlarge"],
    worker_gpu=1,
    compute_purchase_option='spot_with_fallback',
    shutdown_on_close=False
)
client = cluster.get_client()

Output()

/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py:1361: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.3.2  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [3]:
def upload_pkg(client, module, restart=False):
    from pathlib import Path

    dir = Path(module.__file__).parent.parent
    skip = [p for p in os.listdir(dir) if p != module.__name__]
    return client.register_worker_plugin(
        UploadDirectory(dir, skip_words=skip, restart=restart, update_path=True),
        nanny=True,
    )

upload_pkg(client, confirm)
upload_pkg(client, imprint, restart=True)

{}

In [4]:

import dask
@dask.delayed
def check_nvidia_jax():
    import jax
    import subprocess
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    nvidia_smi = result.stdout.decode('ascii')
    jax_platform = jax.lib.xla_bridge.get_backend().platform
    key = jax.random.PRNGKey(0)
    means = jax.random.uniform(key, (10000, 2)).mean(axis=0)
    return nvidia_smi, jax_platform, means

nvidia_smi, jax_platform, means = check_nvidia_jax().compute()
print(nvidia_smi)
print(jax_platform, means)

@dask.delayed
def check_confirm_imprint():
    import confirm
    import imprint
check_confirm_imprint().compute()

Fri Mar 10 14:03:56 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                        On | 00000000:00:1E.0 Off |                    0 |
| N/A   28C    P8                9W /  70W|      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
def setup_worker(worker_args):
    import imprint as ip
    from distributed import get_worker
    ip.package_settings()
    worker = get_worker()
    (model_type, model_args, model_kwargs, algo_type, cfg) = worker_args
    hash_args = hash((
        model_type.__name__,
        model_args,
        tuple(model_kwargs.items()),
        algo_type.__name__,
        tuple(cfg.items()),
    ))
    has_hash = hasattr(worker, 'algo_hash')
    has_algo = hasattr(worker, 'algo')
    if not (has_algo and has_hash) or (has_hash and hash_args != worker.algo_hash):
        model = model_type(*model_args, **model_kwargs)
        cfg['worker_id'] = 2
        worker.algo = algo_type(model, None, None, cfg, None) 
        worker.algo_hash = hash_args
    async def async_process_tiles(tiles_df):
        return await worker.algo.process_tiles(tiles_df = tiles_df)
    import synchronicity
    synchronizer = synchronicity.Synchronizer()
    process_tiles = synchronizer.create(async_process_tiles)[synchronicity.Interface.BLOCKING]
    return process_tiles

@dask.delayed
def dask_process_packet(worker_args, packet_df):
    import jax
    import time
    start = time.time()
    process_tiles = setup_worker(worker_args)
    jax_platform = jax.lib.xla_bridge.get_backend().platform
    assert jax_platform == 'gpu'
    runtime_non_process = time.time() - start
    start = time.time()
    out = process_tiles(packet_df)
    runtime = time.time() - start
    return out, runtime, runtime_non_process

import time
import imprint as ip
import dask
import numpy as np
from confirm.adagrid.calibrate import AdaCalibrate
from confirm.models.wd41 import WD41
from imprint.models.ztest import ZTest1D
NN = 6
g = ip.cartesian_grid(
    theta_min=np.full(4, -1),
    theta_max=np.full(4, 0),
    n=np.full(4, NN),
    null_hypos=[ip.hypo("theta0 > theta1")],
)
init_K = 2**13
K = init_K * 16
g.df['K'] = init_K
cfg = dict(
    init_K = 2**13,
    n_K_double = 4,
    tile_batch_size = 64,
    # lam = -1.96, 
    # delta = 0.01,
    # global_target = 0.001,
    # max_target = 0.01,
    bootstrap_seed=0,
    nB=50,
    alpha=0.025,
    calibration_min_idx=40
)

[worker_id=None] 2023-03-10 09:04:02,637 - imprint.grid - DEBUG 
_gen_short_uuids(n=1296, worker_id=1, t=1678457042, n_bits=18, worker_bits=18) = [4662225207837327360 4662225207837327361 4662225207837327362, ...]:
[worker_id=None] 2023-03-10 09:04:02,639 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-10 09:04:02,641 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-10 09:04:02,643 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-10 09:04:02,654 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-10 09:04:02,733 - imprint.grid - DEBUG 
_gen_short_uuids(n=432, worker_id=1, t=1678457043, n_bits=18, worker_bits=18) = [4662225276556804096 4662225276556804097 4662225276556804098,

In [8]:
# worker_args = (WD41, (0, K), dict(ignore_intersection=True), AdaCalibrate, cfg)
worker_args = (ZTest1D, (0, K), dict(), AdaCalibrate, cfg)
worker_args_future = client.scatter(worker_args, broadcast=True)
start = time.time()
out, inner_runtime, ir = dask_process_packet(worker_args_future, g.df).compute()
runtime1 = time.time() - start
start = time.time()
out, inner_runtime2, ir2 = dask_process_packet(worker_args_future, g.df).compute()
runtime2 = time.time() - start
print(runtime1, inner_runtime, ir)
print(runtime2, inner_runtime2, ir2)

0.32193708419799805 0.06250691413879395 0.001409292221069336
0.2914559841156006 0.07643008232116699 0.0012178421020507812


In [10]:
12.3 / 200.

0.061500000000000006

In [13]:
import asyncio
def dpp():
    out, inner_runtime2, ir2 = dask_process_packet(worker_args_future, g.df).compute()
    return inner_runtime2

async def lots_test():
    ts = [
        asyncio.create_task(asyncio.to_thread(dpp))
        for i in range(1000)
    ]
    return await asyncio.gather(*ts)
    
start = time.time()
await lots_test()
print(time.time() - start)

CancelledError: 

In [14]:
dpp()

CancelledError: dask_process_packet-78d8815b-ec73-478e-8f45-60c99eeb866b

2023-03-10 09:38:48,857 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.61.86:8786' processes=1 threads=4, memory=14.72 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'
ERROR:tornado.application:Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.61.86:8786' processes=1 threads=4, memory=14.72 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-p

[worker_id=None] 2023-03-10 09:38:48,857 - tornado.application - ERROR 
Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.61.86:8786' processes=1 threads=4, memory=14.72 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'


2023-03-10 09:38:53,857 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.61.86:8786' processes=1 threads=4, memory=14.72 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'
ERROR:tornado.application:Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.61.86:8786' processes=1 threads=4, memory=14.72 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-p

[worker_id=None] 2023-03-10 09:38:53,857 - tornado.application - ERROR 
Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.61.86:8786' processes=1 threads=4, memory=14.72 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'


...

[worker_id=None] 2023-03-10 09:39:13,857 - tornado.application - ERROR 
Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.61.86:8786' processes=1 threads=4, memory=14.72 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'


2023-03-10 09:39:15,863 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


[worker_id=None] 2023-03-10 09:39:15,863 - distributed.client - ERROR 
Failed to reconnect to scheduler after 30.00 seconds, closing client


2023-03-10 09:41:28,678 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/comm/tcp.py", line 498, in connect
    stream = await self.client.connect(
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/asyncio/tasks.py", line 456, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python

[worker_id=None] 2023-03-10 09:41:28,678 - distributed.deploy.cluster - WARNING 
Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/comm/tcp.py", line 498, in connect
    stream = await self.client.connect(
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/asyncio/tasks.py", line 456, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/co

In [76]:
2.21 / (g.df.shape[0] * K) * 1e9

1.5011540165653936

In [6]:
import cloudpickle

In [7]:
abcdef = cloudpickle.dumps(g.df)

In [14]:
g.df.memory_usage().sum() / 1e6, out.memory_usage().sum() / 1e6

(0.990128, 5.39)

In [62]:
import logging
logger = logging.getLogger('confirm.cloud.coiled')
class CoiledAlgo:
    def __init__(self, client, worker_args_future, wrapped):
        super().__init__()
        self.client = client
        self.wrapped = wrapped
        self.worker_args_future = worker_args_future

        self.null_hypos = wrapped.null_hypos
        self.cfg = wrapped.cfg
        self.db = wrapped.db
        self.callback = wrapped.callback
        self.Ks = wrapped.Ks
        self.max_K = wrapped.max_K
        self.driver = wrapped.driver
        
    def get_orderer(self):
        return self.wrapped.get_orderer()
        
    async def process_tiles(self, *, tiles_df):
        start = time.time()
        relevant_cols = [
            c for c in tiles_df.columns if c.startswith('theta') or c.startswith('K')
        ]
        out, compute_runtime = dask_process_packet(self.worker_args_future, tiles_df).compute()
        total_runtime = time.time() - start
        # logger.debug('Compute runtime: %s', compute_runtime)
        # logger.debug('Total runtime: %s', total_runtime)
        print('Compute runtime: %s', compute_runtime)
        print('Total runtime: %s', total_runtime)
        return out
        
    async def convergence_criterion(self, zone_id, report):
        return await self.wrapped.convergence_criterion(zone_id, report)

    async def select_tiles(self, zone_id, new_step_id, report, convergence_task):
        return await self.wrapped.select_tiles(zone_id, new_step_id, report, convergence_task)

In [63]:
import contextlib
from confirm.adagrid.backend import LocalBackend
class CoiledBackend(LocalBackend):
    def __init__(self, client, n_zones=1, coordinate_every=5):
        super().__init__(n_zones, coordinate_every)
        self.client = client

    @contextlib.asynccontextmanager
    async def setup(self, algo_type, algo, kwargs):
        algo_entries = [
            'init_K',
            'n_K_double',
            'tile_batch_size',
            'lam',
            'delta',
            'worker_id',
            'global_target',
            'max_target',
            'bootstrap_seed',
            'nB',
            'alpha',
            'calibration_min_idx'
        ]
        filtered_cfg = {k: v for k, v in algo.cfg.items() if k in algo_entries}
        worker_args = (type(algo.driver.model), (algo.cfg['model_seed'], algo.max_K), algo.cfg['model_kwargs'], algo_type, filtered_cfg)
        worker_args_future = client.scatter(worker_args, broadcast=True)
        self.algo = CoiledAlgo(self.client, worker_args_future, algo)
        yield

In [64]:
import numpy as np
import imprint as ip
from imprint.models.ztest import ZTest1D
import confirm.adagrid as ada

d = 2
g = ip.cartesian_grid(
    theta_min=np.full(d, -1),
    theta_max=np.full(d, 0),
    null_hypos=[ip.hypo("theta0 > theta1")],
)
db = ada.ada_calibrate(ZTest1D, g=g, alpha=0.025, prod=False, backend=CoiledBackend(client))

[worker_id=None] 2023-03-09 17:05:21,515 - imprint.grid - DEBUG 
_gen_short_uuids(n=4, worker_id=1, t=1678399525, n_bits=18, worker_bits=18) = [4658272669693902848 4658272669693902849 4658272669693902850, ...]:
[worker_id=None] 2023-03-09 17:05:21,519 - imprint.grid - DEBUG 
_gen_short_uuids(n=4, worker_id=1, t=1678399526, n_bits=18, worker_bits=18) = [4658272738413379584 4658272738413379585 4658272738413379586, ...]:
[worker_id=1] 2023-03-09 17:05:21,604 - confirm.adagrid.init - DEBUG 
Initialized database with 3 tiles and 1 null hypos. The tiles are split between 1 zones with packet_size=1024.
[worker_id=1] 2023-03-09 17:05:22,940 - confirm.adagrid.backend - DEBUG 
init took 1.4137721061706543
[worker_id=1] 2023-03-09 17:05:22,966 - confirm.adagrid.backend - DEBUG 
setup took 0.025721073150634766
[worker_id=1] 2023-03-09 17:05:22,967 - confirm.adagrid.step - DEBUG 
waiting for work: (zone_id=0, step_id=0, packet_id=0)
[worker_id=1] 2023-03-09 17:05:22,969 - confirm.adagrid.step - DEB

In [ ]:
import confirm.models.wd41 as wd41
model = wd41.WD41(0, 1, ignore_intersection=True)
grid = ip.cartesian_grid(
    [-2.5, -2.5, -2.5, -2.5],
    [1.0, 1.0, 1.0, 1.0],
    n=[10, 10, 10, 10],
    null_hypos=model.null_hypos,
)
db = ada.DuckDBTiles.connect('./wd41-4.db')
ada.ada_calibrate(
    wd41.WD41,
    g=grid,
    db=db,
    alpha=0.025,
    bias_target=0.0025,
    grid_target=0.0025,
    std_target=0.005,
    n_K_double=6,
    calibration_min_idx=100,
    step_size=2**18,
    packet_size=2**15,
    model_kwargs={"ignore_intersection": True},
    backend=CoiledBackend(client)
)

[worker_id=None] 2023-03-09 17:05:40,062 - imprint.grid - DEBUG 
_gen_short_uuids(n=10000, worker_id=1, t=1678399545, n_bits=18, worker_bits=18) = [4658274044083437568 4658274044083437569 4658274044083437570, ...]:
[worker_id=None] 2023-03-09 17:05:40,066 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 17:05:40,071 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 17:05:40,079 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 17:05:40,120 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 17:05:40,137 - imprint.grid - DEBUG 
_gen_short_uuids(n=2000, worker_id=1, t=1678399546, n_bits=18, worker_bits=18) = [4658274112802914304 4658274112802914305 465827411280291430

CancelledError: dask_process_packet-b61fc584-3675-48cb-a027-edcad39ffdb0

2023-03-09 21:21:25,772 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.50.36:8786' processes=1 threads=4, memory=14.71 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'
ERROR:tornado.application:Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.50.36:8786' processes=1 threads=4, memory=14.71 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-p

[worker_id=None] 2023-03-09 21:21:25,772 - tornado.application - ERROR 
Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.50.36:8786' processes=1 threads=4, memory=14.71 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'


2023-03-09 21:21:30,772 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.50.36:8786' processes=1 threads=4, memory=14.71 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'
ERROR:tornado.application:Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.50.36:8786' processes=1 threads=4, memory=14.71 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-p

[worker_id=None] 2023-03-09 21:21:30,772 - tornado.application - ERROR 
Exception in callback <bound method Client._heartbeat of <Client: 'tls://10.0.50.36:8786' processes=1 threads=4, memory=14.71 GiB>>
Traceback (most recent call last):
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py", line 1445, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
AttributeError: 'NoneType' object has no attribute 'send'


/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py:1361: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.3.2  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [58]:
client.close()
cluster.close()

In [ ]:
import dask

@dask.delayed
def run_ada():
    import numpy as np
    import imprint as ip
    from imprint.models.ztest import ZTest1D
    import confirm.adagrid as ada
    ip.package_settings()
    d = 2
    g = ip.cartesian_grid(
        theta_min=np.full(d, -1),
        theta_max=np.full(d, 0),
        null_hypos=[ip.hypo("theta0 > theta1")],
    )
    db = ada.ada_validate(
        ZTest1D,
        g=g,
        lam=-1.96,
        prod=False
    )
    return ip.Grid(db.get_results(), 1).prune_inactive().n_tiles

NN = run_ada().compute()
NN

355